<a href="https://colab.research.google.com/github/sainikhila11/SaiNikhila_INFO5731_Spring2024/blob/main/Yavanamanda_Sai_Exercise_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 In-class Exercise 5**

**This exercise aims to provide a comprehensive learning experience in text analysis and machine learning techniques, focusing on both text classification and clustering tasks.**

***Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks***.

**Expectations**:
*   Students are expected to complete the exercise during lecture period to meet the active participation criteria of the course.
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

**Total points**: 40

**Deadline**: This in-class exercise is due at the end of the day tomorrow, at 11:59 PM.

**Late submissions will have a penalty of 10% of the marks for each day of late submission, and no requests will be answered. Manage your time accordingly.**


## **Question 1 (20 Points)**

The purpose of the question is to practice different machine learning algorithms for **text classification** as well as the performance evaluation. In addition, you are requried to conduct **10 fold cross validation** (https://scikit-learn.org/stable/modules/cross_validation.html) in the training.



The dataset can be download from canvas. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data.


**Algorithms:**

*   MultinominalNB
*   SVM
*   KNN
*   Decision tree
*   Random Forest
*   XGBoost
*   Word2Vec
*   BERT

**Evaluation measurement:**


*   Accuracy
*   Recall
*   Precison
*   F-1 score


In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Load data
with open("stsa-train.txt", "r") as f:
    lines = f.readlines()
train_data = pd.DataFrame({"review": lines})

with open("stsa-test.txt", "r") as f:
    lines = f.readlines()
test_data = pd.DataFrame({"review": lines})

# Preprocessing
train_data['sentiment'] = train_data['review'].apply(lambda x: int(x.split()[0]))
train_data['review'] = train_data['review'].apply(lambda x: ' '.join(x.split()[1:]))
test_data['sentiment'] = test_data['review'].apply(lambda x: int(x.split()[0]))
test_data['review'] = test_data['review'].apply(lambda x: ' '.join(x.split()[1:]))

# Splitting into training and validation data
X_train, X_val, y_train, y_val = train_test_split(train_data['review'], train_data['sentiment'], test_size=0.2, random_state=42)

# Create TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit TF-IDF vectorizer on training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transform validation and test data using the fitted vectorizer
X_val_tfidf = tfidf_vectorizer.transform(X_val)
X_test_tfidf = tfidf_vectorizer.transform(test_data['review'])

# Define algorithms
models = {
    "MultinomialNB": MultinomialNB(),
    "SVM": SVC(),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "XGBoost": XGBClassifier()
}

# Define evaluation metrics
metrics = {
    "Accuracy": accuracy_score,
    "Recall": recall_score,
    "Precision": precision_score,
    "F1 Score": f1_score
}

# Perform 10-fold cross-validation and evaluation
for name, model in models.items():
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    cv_results = cross_val_score(model, X_train_tfidf, y_train, cv=kfold, scoring='accuracy')
    print(f"{name}:")
    print(f"Mean Accuracy: {cv_results.mean()}")
    print(f"Standard Deviation: {cv_results.std()}")

# Select the best performing model and evaluate on validation set
best_model_name = max(models, key=lambda x: cross_val_score(models[x], X_train_tfidf, y_train, cv=kfold, scoring='accuracy').mean())
best_model = models[best_model_name]
best_model.fit(X_train_tfidf, y_train)
y_pred = best_model.predict(X_val_tfidf)

print(f"Best Model: {best_model_name}")
for metric_name, metric_func in metrics.items():
    print(f"{metric_name}: {metric_func(y_val, y_pred)}")

# Evaluate on test set
test_predictions = best_model.predict(X_test_tfidf)
test_accuracy = accuracy_score(test_data['sentiment'], test_predictions)
print(f"Test Accuracy: {test_accuracy}")


MultinomialNB:
Mean Accuracy: 0.7796218199385041
Standard Deviation: 0.016677791563826178
SVM:
Mean Accuracy: 0.775101350689707
Standard Deviation: 0.019039419120688787
KNN:
Mean Accuracy: 0.7156788374537312
Standard Deviation: 0.013469805944067489
Decision Tree:
Mean Accuracy: 0.6029598971151775
Standard Deviation: 0.009781428305199145
Random Forest:
Mean Accuracy: 0.7093520084083534
Standard Deviation: 0.01067975277848651
XGBoost:
Mean Accuracy: 0.6954472160385425
Standard Deviation: 0.013534680121139722
Best Model: MultinomialNB
Accuracy: 0.7933526011560693
Recall: 0.8906030855539971
Precision: 0.7532621589561092
F1 Score: 0.8161953727506427
Test Accuracy: 0.8034047226798462


## **Question 2 (20 Points)**

The purpose of the question is to practice different machine learning algorithms for **text clustering**.

Please downlad the dataset by using the following link.  https://www.kaggle.com/PromptCloudHQ/amazon-reviews-unlocked-mobile-phones
(You can also use different text data which you want)

**Apply the listed clustering methods to the dataset:**
*   K-means
*   DBSCAN
*   Hierarchical clustering
*   Word2Vec
*   BERT

You can refer to of the codes from  the follwing link below.
https://www.kaggle.com/karthik3890/text-clustering

In [2]:
!pip install sentence-transformers


In [7]:
# Step 1: Mount Google Drive to access files (only if working in Google Colab)
from google.colab import drive


# Step 2: Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from gensim.models import Word2Vec
from sentence_transformers import SentenceTransformer

# Step 3: Load the dataset
data_path = "/content/imdb_reviews_sentimentanalysis.csv"  # Update with your file path
df = pd.read_csv(data_path)

# Step 4: Explore the dataset
print(df.head())

# Step 5: Preprocess the data if necessary

# Step 6: Text Clustering with K-means
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, min_df=5, stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['clean_text'].values.astype('U'))

# K-means Clustering
num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters, n_init=10, max_iter=300, random_state=42)
clusters_kmeans = kmeans.fit_predict(tfidf_matrix)

# Step 7: Text Clustering with DBSCAN
# DBSCAN Clustering
dbscan = DBSCAN(eps=0.5, min_samples=5)
clusters_dbscan = dbscan.fit_predict(tfidf_matrix)

# Step 8: Text Clustering with Hierarchical Clustering
# Hierarchical Clustering
agglomerative = AgglomerativeClustering(n_clusters=num_clusters, linkage='ward')
clusters_hierarchical = agglomerative.fit_predict(tfidf_matrix.toarray())

# Step 9: Text Clustering with Word2Vec
# Tokenization
sentences = [review.split() for review in df['clean_text'].values]

# Training Word2Vec model
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Applying K-means clustering on Word2Vec embeddings
word_vectors = word2vec_model.wv
word_vectors_matrix = word_vectors.vectors
kmeans_word2vec = KMeans(n_clusters=num_clusters, random_state=42)
clusters_word2vec = kmeans_word2vec.fit_predict(word_vectors_matrix)

# Step 10: Text Clustering with BERT
# Sentence embeddings with BERT
model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_embeddings = model.encode(df['clean_text'], show_progress_bar=True)

# Applying K-means clustering on BERT embeddings
kmeans_bert = KMeans(n_clusters=num_clusters, random_state=42)
clusters_bert = kmeans_bert.fit_predict(sentence_embeddings)

# Step 11: Visualize the clustering results (optional)

# Step 12: Analyze the clustering results (optional)

# Step 13: Print out the cluster assignments for each method
print("K-means Clustering:")
print(clusters_kmeans)
print("DBSCAN Clustering:")
print(clusters_dbscan)
print("Hierarchical Clustering:")
print(clusters_hierarchical)
print("Word2Vec Clustering:")
print(clusters_word2vec)
print("BERT Clustering:")
print(clusters_bert)


   document_id                                         clean_text Sentiment 
0            1  margot does the best with what shes given but ...   negative
1            2  before making barbie greta gerwigsinglehandedl...   positive
2            3  the quality the humor and the writing of the m...   negative
3            4  as much as it pains me to give a movie called ...   positive
4            5  as a woman that grew up with barbie i was very...   negative


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

K-means Clustering:
[4 1 3 4 1 1 1 4 4 1 0 0 1 1 4 3 1 1 2 1 3 1 1 4 1 4 1 3 4 1 1 1 4 4 1 2 0
 1 1 4 3 1 1 2 1 3 1 1 4 1 4 1 3 4 1 1 1 4 4 1 1 0 1 1 4 3 1 1 2 1 3 1 1 4
 1 4 1 3 4 1 1 1 4 4 1 4 0 1 1 4 3 1 1 2 1 3 1 1 4 1]
DBSCAN Clustering:
[ 0 -1 -1 -1 -1 -1 -1 -1 -1 -1  1  1 -1  2 -1 -1  3 -1  4 -1 -1 -1 -1 -1
 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1  4  1 -1  2 -1 -1  3 -1  4 -1 -1 -1 -1
 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1  3  1 -1  2 -1 -1  3 -1  4 -1 -1 -1
 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1  0  1 -1  2 -1 -1  3 -1  4 -1 -1
 -1 -1 -1  2]
Hierarchical Clustering:
[2 0 2 1 1 0 1 1 1 1 0 0 1 0 1 1 3 1 4 1 0 0 0 2 1 2 0 2 1 1 0 1 1 1 1 4 0
 1 0 1 1 3 1 4 1 0 0 0 2 1 2 0 2 1 1 0 1 1 1 1 3 0 1 0 1 1 3 1 4 1 0 0 0 2
 1 2 0 2 1 1 0 1 1 1 1 2 0 1 0 1 1 3 1 4 1 0 0 0 2 0]
Word2Vec Clustering:
[1 1 1 ... 3 3 3]
BERT Clustering:
[3 1 0 1 1 4 4 3 2 1 0 0 2 1 1 3 1 2 4 3 3 1 1 2 2 3 1 0 1 1 4 4 3 2 1 4 0
 2 1 1 3 1 2 4 3 3 1 1 2 2 3 1 0 1 1 4 4 3 2 1 1 0 2 1 1 3 1 2 4 3 3 1 1 2
 2 3 1 0 1 1 4 4

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


**In one paragraph, please compare the results of K-means, DBSCAN, Hierarchical clustering, Word2Vec, and BERT.**


Each clustering method, including K-means, DBSCAN, Hierarchical clustering, Word2Vec, and BERT, offers unique approaches to understanding textual data. K-means partitions the data into clusters based on similarity, but its performance can be sensitive to the choice of initial centroids and the number of clusters. DBSCAN, on the other hand, identifies clusters based on density, capable of handling irregularly shaped clusters and noise points. Hierarchical clustering constructs a hierarchical tree of clusters, providing insights into the hierarchical structure of the data. Word2Vec and BERT embeddings capture semantic similarities, enabling clustering based on semantic context. Word2Vec clusters individual words, while BERT clusters entire sentences, providing more contextually rich representations. Ultimately, the choice of clustering method depends on the specific characteristics of the data and the analytical goals, with each method offering its strengths and limitations in uncovering meaningful patterns in text data.







# Mandatory Question

Working on the 10 fold validation kind of dataset is something new i did with this exercise and i have got to know about new algorithms like Multinomial NB.

